## Import Necessary Packages

In [122]:
%reset

In [123]:
# API Packages
import requests

# Data Science Packages
import numpy as np
import pandas as pd
import scipy

# Time Series Packages
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.seasonal import STL
from statsmodels.tsa.arima.model import ARIMA

# Visualization Packages
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

## Acquire and Parse Stock Daily Trade Data

#### Alphavantage API is used to acquire historical daily closing prices of a particular stock. Stock prices are considered random walks without patterns to their fluctuations, so an initial shorter period of 91 days is used for this analysis.

In [124]:
# Inputs
symbol = 'AAPL' # Apple ticker example
days = 91 # Historical days desired for analysis (most recent --> 91 days past)

In [125]:
# Get your free API Key from https://www.alphavantage.co/
# API Key stored in config file
import config
api_key = config.api_key

In [126]:
# Use https://www.alphavantage.co as the API to pull BTC from
url = f'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED&symbol={symbol}&outputsize=full&earnings&apikey={api_key}'
r = requests.get(url) # Requests pulls from the API url
data = r.json() # Loads data into dictionary objects (meta data + daily trading)

In [127]:
# Parse the data from the API into a Pandas DataFrame
my_columns = ['Date', 'Close'] # Create columns
df = pd.DataFrame(columns=my_columns) # Instantiate a DataFrame
for row in data['Time Series (Daily)']: # Loop through every dictionary line
    df = pd.concat([df,
        pd.DataFrame(
            [   
                [row, # Date
                data['Time Series (Daily)'][row]['4. close']], # Close Price (USD)
            ], columns=my_columns # Create column names
        )], ignore_index=True, axis=0 # Formatting purposes
    )

df = df.loc[::-1].reset_index(drop=True) # Reverse DataFrame's order so it's sequential
df = df.iloc[-days:-1,:] # # Enter number of historical days for analysis
df['Close'] = df[['Close']].astype('float') # Change str to float
df['Close']=df['Close'] # Formula for "daily returns"
df['Close']=df['Close'].fillna(0) # Fill na values with 0

## Verify there are no missing values by visualizing data

In [128]:
# Plot closing price time series as line plot
fig=px.line(df, x='Date', y='Close', title='Time Series Plot: Closing Price')
fig.update_layout(autotypenumbers='convert types')
fig.show()

# Describe statistics of numerical values only
df['Close'].describe()

count     90.000000
mean     145.226889
std        8.679827
min      125.020000
25%      139.647500
50%      146.250000
75%      150.995000
max      163.430000
Name: Close, dtype: float64

## Stationarity Test

#### Since stock prices are not stationary, the 'd' component of the ARIMA model will need to be adjusted

In [129]:
# Perform Augmented Dickey-Fuller test to determine if the time-series has stationarity
# P-value > 0.05 suggests a lack of stationarity
adf = adfuller(df['Close'])
print('Test statistic:', adf[0])
if adf[1] > 0.05:
    print('P-value:', adf[1], '> 0.05, so the data does not have stationarity')
else:
    print('P-value:', adf[1], '< 0.05, so the data has stationarity')

Test statistic: -1.7173234410567089
P-value: 0.4222097455163719 > 0.05, so the data does not have stationarity


## Split the data into training / testing data with a percentage split

In [130]:
split_percent = 0.95 # What percentage the data is to be split (0.9=90%)

split = int(np.floor(len(df['Close'])*split_percent)) # Day to split at
training_data = df['Close'][:split] # From day 0 to split
testing_data = df['Close'][split:] # From split day to end of data

# Visualize the split testing / training data
fig = go.Figure()
fig.add_trace(go.Line(x=df['Date'][training_data.index], y=training_data, name='Training Data'))
fig.add_trace(go.Line(x=df['Date'][testing_data.index], y=testing_data, name='Testing Data'))
fig.update_layout(title='Test/Train Split Visualization', xaxis_title='Days', yaxis_title='Revenue')
fig.show()


c:\Users\qmeye\OneDrive\Documents\WGU\D208\NBM2\churn\venv\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




### View 1 lag applied to the time series

In [131]:
df_tf=df['Close'].copy() # Create copied dataframe
df_tf=df_tf-df_tf.shift(1) # Apply difference transform (lag 1)
df_tf = pd.concat([df_tf, df['Date']], axis=1)
df_tf.dropna(inplace=True) # Remove infinite and nan values
fig=px.line(df_tf, x='Date', y='Close', title='1 Lag Transformed Time Series Plot') # Plot new transformed time-series
fig.show()

# Conduct Augmented Dickey-Fuller test
adf = adfuller(df_tf['Close'])
print('Test statistic:', adf[0])
print('P-value:', adf[1], '< 0.05, so the data is stationary with a lag value of 1')

Test statistic: -7.936152573142065
P-value: 3.4093243196025943e-12 < 0.05, so the data is stationary with a lag value of 1


### Lag 1 used to transform the data to stationary means that d=1 for ARIMA.

### Plot the autocorrelation (ACF) and partial autocorrelation (PACF)

In [132]:
nlags=20 # specify number of lags to plot

# Conduct the ACF
df_acf, confint = acf(df['Close'], nlags=nlags, alpha=0.05) # Use sm's acf
fig=go.Figure() # create figure
upper_bounds = df_acf-confint[:,0]
lower_bounds = df_acf-confint[:,1]

 # Plot ACF
fig.add_trace(go.Scatter(
    x=np.arange(nlags+1),
    y=df_acf, name='ACF'))
fig.update_layout(title='Autocorrelation Function',
    xaxis_title='Lags', yaxis_title='Correlation')
# Add upper and lower 95% confidence bounds
fig.add_trace(go.Line(
    x=np.arange(nlags+1),
    y=upper_bounds, line_color="#ff0000", name='Outer Bounds'))
fig.add_trace(go.Line(
    x=np.arange(nlags+1),
    y=lower_bounds, line_color="#ff0000", fill='tonexty', name='95% Confidence Interval'))
fig.show()

# Conduct the PACF
df_pacf, confint = pacf(df['Close'], nlags=nlags, alpha=0.05) # Use sm's acf
fig=go.Figure() # create figure
upper_bounds = df_pacf-confint[:,0]
lower_bounds = df_pacf-confint[:,1]

# Plot PACF
fig.add_trace(go.Scatter(
    x=np.arange(nlags),
    y=df_pacf, name='PACF'))
fig.update_layout(title='Passive Autocorrelation Function',
    xaxis_title='Lags', yaxis_title='Correlation')
# Add upper and lower 95% confidence bounds
fig.add_trace(go.Line(
    x=np.arange(nlags+1),
    y=upper_bounds, line_color="#ff0000", name='Outer Bounds'))
fig.add_trace(go.Line(
    x=np.arange(nlags+1),
    y=lower_bounds, line_color="#ff0000", fill='tonexty', name='95% Confidence Interval'))
fig.show()

c:\Users\qmeye\OneDrive\Documents\WGU\D208\NBM2\churn\venv\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### For the Moving Average (MA) model, q is chosen by assessing the sharp cut-off point after laq q. There is a sharp cut-off after lag 1 in the ACF plot where spikes lie outside the 95% confidence interval, so q=1 for the ARIMA model. (10)

##### For the Autoregressive (AR) model, p is chosen based on the number of significant spikes from the PACF plot. There are 5 significant spike outside the 95% confidence interval, so p=5 for the ARIMA model. (10)

### Look for seasonal effects on revenue, trends, decomposed time series, and confirmation
###### (1)

In [133]:
# Seasonal-Trend Decomposition
period=5 # There are 5 days in a week of the open stock market
stl = STL(df['Close'], period=period) # Conduct the Seasonal-Trend Decomposition
result = stl.fit() # Fit the data to an object
seasonal, trend, resid = result.seasonal, result.trend, result.resid # Get the results for different plots


# Plot the Seasonal-Trend Decomposition (Decomposed Time Series, Trend, and Seasonal effects)
fig = make_subplots(rows=4, cols=1,
    subplot_titles=('Original Series', 'Trend', 'Seasonal', 'Residual'))
fig.add_trace(go.Line(x=df['Date'], y=df['Close'], name='Original'), # Plot original
    row=1, col=1)
fig.add_trace(go.Line(x=df['Date'], y=trend, name='Trend'), # Plot trend
    row=2, col=1)
fig.add_trace(go.Line(x=df['Date'], y=seasonal, name='Seasonal'), # Plot seasonal component
    row=3, col=1)
fig.add_trace(go.Line(x=df['Date'], y=resid, name='Residual'), # Plot residuals
    row=4, col=1)
fig.update_xaxes(title_text='Date', row=4, col=1) # Update xaxis
fig.update_yaxes(title_text='Close', row=3, col=1) # Update yaxis

# Add vertical lines to represent chunks of each season
days = np.arange(np.min(len(df['Date'])), np.max(len(df['Date'])), period)
for day in days:
    fig.add_vline(x=day, line_dash='dash')
fig.show()

# Confirmation
estimated = seasonal + trend # Trend + Seasonal + Residuals is fitted from original model
# Plot (Trend + Seasonal) = Estimated vs Original Series
fig2 = px.line(x=df['Date'], y=estimated)
fig2 = fig2.add_trace(go.Line(x=df['Date'], y=estimated, name='Predicted'))
fig2.add_trace(go.Line(x=df['Date'], y=df['Close'], name='Original'))
fig2.update_layout(
    title='Original Series vs Predicted (Seasonal + Trend) Series',
    xaxis_title='Date', yaxis_title='Close')
fig2.show()

c:\Users\qmeye\OneDrive\Documents\WGU\D208\NBM2\churn\venv\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




##### One can see from the above two graphs that there is clearly no seasonal component to the data. Additionally, there are no trends in the residuals of the decomposed time series.

### Power Spectral Density 
###### (3)

In [134]:
# Use scipy to find the periodogram to find the PSD
(f, S) = scipy.signal.periodogram(df['Close'], len(df), scaling='density')
df_psd = pd.DataFrame({'f':f, 'S':S})
fig=px.line(df_psd, x=f, y=S)
fig.update_layout(
    title='Power Spectrum Density - Welch\'s Method',
    xaxis_title='Frequency (Days)', yaxis_title='PSD')
fig.show()

##### The PSD shows the greatest amplitude at Frequency = 2 day. This further confirms that there are no hidden patterns in the data, or in other words 'seasonality'.

## Autoregressive Integrated Moving Average (ARIMA)
##### Previously it was discovered that a good fit for ARIMA's (p, d, q) = (1, 1, 5). The 'd' variable applys lag 1 to the data which is why it was not cleaned earlier.

In [135]:
# Max prediction day:
max_day=len(df['Date'])

# Fit the ARIMA model using statsmodel
model = ARIMA(training_data, order=(2,1,7)).fit()
prediction_model = model.get_prediction(0, max_day-1)
pred_vals = prediction_model.predicted_mean
pred_vals = pred_vals.replace(0,pred_vals.iloc[1])

# 95% confidence interval from the predictions
confint=prediction_model.conf_int(alpha=0.05)

# Plot the ARIMA Model's predictions with confidence interval (only for predicted future values)
xaxis=np.array(range(0,max_day+1))
fig = go.Figure()
fig.add_trace(go.Line(x=df['Date'], y=df['Close'], name='Original'))
fig.add_trace(go.Line(x=df['Date'], y=pred_vals, name='Predicted Values'))
fig.add_trace(go.Line(x=df['Date'][split:], y=confint['upper Close'][split:],
     line=dict(color="#b9b9b9"), name='Upper 95% Confidence'))
fig.add_trace(go.Line(x=df['Date'][split:], y=confint['lower Close'][split:],
     line=dict(color="#b9b9b9"), name='Lower 95% Confidence', fill='tonexty'))
fig.update_layout(title=f'{symbol} Daily Price Forecast using ARIMA', xaxis_title='Days', yaxis_title='Closing Price (USD)')
fig.show()

# Print the summary of the model as part of the project
print(model.summary())

mape = np.mean(np.abs((df['Close'][split:]-pred_vals)/df['Close'][split:]))
print('MAPE =', mape*100,'%')

c:\Users\qmeye\OneDrive\Documents\WGU\D208\NBM2\churn\venv\lib\site-packages\statsmodels\base\model.py:604: ConvergenceWarning:

Maximum Likelihood optimization failed to converge. Check mle_retvals

c:\Users\qmeye\OneDrive\Documents\WGU\D208\NBM2\churn\venv\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




                               SARIMAX Results                                
Dep. Variable:                  Close   No. Observations:                   85
Model:                 ARIMA(2, 1, 7)   Log Likelihood                -219.928
Date:                Mon, 09 Jan 2023   AIC                            459.856
Time:                        14:35:05   BIC                            484.164
Sample:                             0   HQIC                           469.628
                                 - 85                                         
Covariance Type:                  opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1          0.0287      0.162      0.177      0.859      -0.289       0.346
ar.L2         -0.8829      0.120     -7.352      0.000      -1.118      -0.648
ma.L1         -0.0677      0.199     -0.340      0.7

# Conclusions:
* Stock prices are essentially random walks and cannot be accurately modeled with ARIMA solely based off of pre-existing price data
* There is little to no seasonality with stock prices
* ARIMA models can vary stock to stock, or based on historical days used to model

# Further areas to research:
* Comparing autoarima to the manual method of finding (q, d, p)
* Look at a portfolio of stocks to determine the MAPE of each projection and assess gains / losses if actually trading
* Create an app that updates buy / sell based off of ARIMA modeling of stocks in a portfolio using rolling window ARIMA method